In [ ]:
#########################################################################
###########------- Deep Learning Inmersion  ------------###########################
#########################################################################
# Capacitador: André Omar Chávez Panduro
# email: andre.chavez@urp.edu.pe
# Tema : AutoEncoders - AE
# version: 2.0
#########################################################################

In [ ]:
# Dataset tomado de: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
# Nos vinculamos a nuestro drive, para poder usar facilmente GColab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. Leemos los datos e instalamos las librerias necesarias!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Importamos el set de datos!
datos = pd.read_csv('',sep=",")

In [ ]:
datos.shape

In [1]:
# Visualizamos los primeros valores de los datos!
datos.head(5)

In [1]:
# Revisamos las distribucion de las clases en el target!
nr_clases = datos['Class'].value_counts(sort=True)
print(nr_clases)
# Tenemos 492    clientes fraudulentos --> 1
# Tenemos 284315 clientes no fraudulentos --> 0

In [ ]:
# 2. Analisis Exploratorio de Datos - AED

In [1]:
# Cantidad de registros normales vs. fraudulentos
nr_clases.plot(kind = 'bar', rot=0)
plt.xticks(range(2), ['Normales', 'Fraudulentos'])
plt.title("Distribución de los datos")
plt.xlabel("Clase")
plt.ylabel("Cantidad")
plt.show()

In [ ]:
# Conclusion:
# Estamos frente a un problema completamente desbalanceado!

In [1]:
# Monto de las transacciones vs. tiempo
normales = datos[datos.Class==0]
fraudulentos = datos[datos.Class==1]
plt.scatter(normales.Time/3600, normales.Amount, 
	alpha = 0.5, c='#19323C', label='Normales', s=3)
plt.scatter(fraudulentos.Time/3600, fraudulentos.Amount, 
	alpha = 0.5, c='#F2545B', label='Fraudulentos', s=3)
plt.xlabel('Tiempo desde la primera transacción (h)')
plt.ylabel('Monto (Euros)')
plt.legend(loc='upper right')
plt.show()
# Conclusiones:
# Los fraudulentos hacen pocas  trx pero con montos muy grandes.
# Los fraudulentos hacen muchas trx pero con montos muy pequeños.

In [1]:
# Distribución de las características V1 a V28 en normales y fraudulentos
import matplotlib.gridspec as gridspec
import seaborn as sns

v_1_28 = datos.iloc[:,0:30].columns
gs = gridspec.GridSpec(28, 1)
for i, cn in enumerate(datos[v_1_28]):
    sns.distplot(datos[cn][datos.Class == 1], bins=50, 
    	label='Fraudulentos', color='#F2545B')
    sns.distplot(datos[cn][datos.Class == 0], bins=50, 
    	label='Normales', color='#19323C')
    plt.xlabel('')
    plt.title('Histograma característica: ' + str(cn))
    plt.legend(loc='upper right')
    plt.show()
    # Conclusiones:

In [ ]:
datos.head(3)

In [ ]:
# 3. Pre procesamiento y Tratamiento de Datos!
# Como las variables son resultado de un ACP, ya están escalados e incorrelacionadas linealmente.

In [ ]:
# La variable "Tiempo" no aporta información. La eliminaremos
from sklearn.preprocessing import StandardScaler
datos.drop(['Time'], axis=1, inplace=True) # Eliminamos la variable tiempo!
datos['Amount'] = StandardScaler().fit_transform(datos['Amount'].values.reshape(-1,1))

In [1]:
# Para la entrada a la Red Neuronal , los datos deben ser escalados!
datos.head()

In [ ]:
# Divido los datos en train/test 
from sklearn.model_selection import train_test_split
# Data de entrenamiento!
X_train, X_test = train_test_split()

In [ ]:
# X_train, X_test tienen las covariables y el target!
# X_train   --- > Y (Clase: No Fraudes y Fraudes)
# X_test   --- >  Y (Clase: No Fraudes y Fraudes)

In [ ]:
# train_test_split (X,y,%test,estratificado,semilla) --> X_train, y_train

In [ ]:
# Me quedo con una sola clase!
X_train = X_train[X_train.Class == 0] # Me quedo con los no fraudes !
# Solo estoy quedandome con los no fraudes de los datos de entrenamiento!
# X_train   --- > Y (Clase: No Fraudes)

In [1]:
X_train.Class.value_counts()

In [ ]:
# Solo tienen 0 o No Fraudulentos!
X_train = X_train.drop(['Class'], axis=1) # Le quito el target! Solo X, ademas solo tenemos no fraudes.
X_train = X_train.values # Lo guardo como valores!

In [ ]:
# Data de test!
Y_test = X_test['Class']                # Target de test Y
X_test = X_test.drop(['Class'], axis=1) # Covariables de test X pero de Fraudulentos y No Fraudulentos!
X_test = X_test.values
# X_test   --- >  Y (Clase: No Fraudes y Fraudes)

In [ ]:
# Hasta aqui esta claro?


In [ ]:
# 4. Creamos el AutoEncoder : 29-20-14-20-29, tanh-relu-tanh-relu

In [ ]:
import numpy as np
np.random.seed(5)
from keras.models import Model, load_model
from keras.layers import Input, Dense

In [1]:
X_train.shape[1]

In [ ]:
dim_entrada = X_train.shape[1] # Cantidad de features! 29 X's
capa_entrada = Input(shape=(dim_entrada,)) # Configuras un Input con capacidad para que albergue todas tus X's

In [ ]:
# Creamos dentro del AE: Encoder!

In [ ]:
encoder = Dense(20, activation='tanh')(capa_entrada) # 1 ra Capa Densa en el Encoder (Capa de Entrada)
encoder = Dense(14, activation='relu')(encoder)      # 2 da Capa Densa en el Encoder

# Ya tenemos o definimos un Encoder!

In [ ]:
# Creamos dentro del AE: Decoder!

In [ ]:
# El dataset original tenia 400 variables , por ACP se comprimieron a 28 CP.
# Le pongo 20 neuronas en la capa oculta para tratar de aprender lo mejor posible!

In [ ]:
decoder = Dense(20, activation='tanh')(encoder)      # 1 ra Capa Densa en el Decoder
decoder = Dense(29, activation='relu')(decoder)      # 2 da Capa Densa en el Decoder , capa final!
# El decoder debe tener en su capa final la misma cantidad de neuronas que 
# el numero de variables iniciales o features!

In [ ]:
# Creamos el AE (*)
autoencoder = Model(inputs=capa_entrada, outputs=decoder)

In [ ]:
# Elegimos el optimizador y la funcion de costo!
from keras.optimizers import SGD
sgd = SGD(lr=0.01)
autoencoder.compile(optimizer='sgd', loss='mse') # ¿Porque el mse?
# Pero el problema no es de clasificacion?

In [ ]:
# Ajustamos o entrenamos el AE!

In [1]:
nits = 5 # Debemos minimo tener 100
tam_lote = 32
autoencoder.fit(X_train, 
                X_train,
                epochs=nits, batch_size=tam_lote, shuffle=True, validation_data=(X_test,X_test), verbose=1)

# Le voy a ingresar covariables y el debe predecir covariables!
# El autoencoder va entender o aprender las caracteristicas de personas no fraudulentas,
# y va tratar de reconstruirlas.

In [ ]:
# 5. Validamos el AE!

In [1]:
# Predicción X_test -> Autoencoder -> X_pred
X_pred = autoencoder.predict(X_test) # Predigo para fraudes y no fraudes!
ecm = np.mean(np.power(X_test-X_pred,2), axis=1) # Calculamos el ECM para todos los elementos!
# Si hay errores grandes, sospecho de que la prediccion esta muy alejada del valor, lo cual no puede ser!
print(X_pred.shape)

In [1]:
ecm.shape

In [1]:
# Gráfica precision-recall para determinar el umbral
from sklearn.metrics import confusion_matrix, precision_recall_curve
precision, recall, umbral = precision_recall_curve(Y_test, ecm)

plt.plot(umbral, precision[1:], label="Precision",linewidth=5)
plt.plot(umbral, recall[1:], label="Recall",linewidth=5)
plt.title('Precision y Recall para diferentes umbrales')
plt.xlabel('Umbral')
plt.ylabel('Precision/Recall')
plt.legend()
plt.show()



In [1]:
# 5.3. Matriz de confusión
umbral_fijo = 0.75 # Umbral de error!
Y_pred = [1 if e > umbral_fijo else 0 for e in ecm]

conf_matrix = confusion_matrix(Y_test, Y_pred)
print(conf_matrix)

In [1]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)


In [1]:
# Especificidad o correcta clasificacion sobre la clase de no fraudes!
80758 /(80758 + 13061)


In [1]:
# Sensibilidad, recall o verdadero positivo o correcta clasificacion sobre la clase de fraudes!
154/(14 + 154)

In [ ]:
# Podemos probar para distintos umbrales (Reentrenando el AE) y puntos de corte!

In [ ]:
# FIN !